In [1]:
%reset -f
def get_api_key():
    import os
    from dotenv import load_dotenv
    # Unset OPENAI_API_KEY if it exists
    os.environ.pop('OPENAI_API_KEY', None)
    # Provide path and ignore currently set environment variable
    dotenv_path = '.env'
    load_dotenv(dotenv_path=dotenv_path, override=True)
    # Access the environment variable
    api_key = os.getenv('OPENAI_API_KEY')
    if api_key is None:
        print("API_KEY not found.")
    return api_key

In [2]:
## Memory
import openai
openai.api_key = get_api_key()

def get_completion(prompt, llm_model):
    client = openai.OpenAI()
    completion = client.chat.completions.create(
      model=llm_model,
      messages=[
        {"role": "user", "content": prompt}
      ]
    )
    # print(completion)
    return completion.choices[0].message.content

llm_model = "gpt-3.5-turbo"

In [4]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(temperature=0.0, model=llm_model,openai_api_key=openai.api_key)

from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=chat,
    memory = memory,
    verbose=True
)

In [5]:
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=chat,
    memory = memory,
    verbose=True
)
conversation.predict(input="Hi, my name is Bob")
conversation.predict(input="what is 1+1")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Bob
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Bob
AI: Hello Bob! It's nice to meet you. How can I assist you today?
Human: what is 1+1
AI:

> Finished chain.


'1+1 is equal to 2.'

In [6]:
conversation.predict(input="what is my name")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Bob
AI: Hello Bob! It's nice to meet you. How can I assist you today?
Human: what is 1+1
AI: 1+1 is equal to 2.
Human: what is my name
AI:

> Finished chain.


'Your name is Bob.'

In [7]:
print(memory.buffer)

Human: Hi, my name is Bob
AI: Hello Bob! It's nice to meet you. How can I assist you today?
Human: what is 1+1
AI: 1+1 is equal to 2.
Human: what is my name
AI: Your name is Bob.


In [8]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Bob\nAI: Hello Bob! It's nice to meet you. How can I assist you today?\nHuman: what is 1+1\nAI: 1+1 is equal to 2.\nHuman: what is my name\nAI: Your name is Bob."}

In [9]:
# Refresh memory

# LLM models are stateless, each transaction is independent

# Chatbots appear to have memory by providing the full conversation as context
memory = ConversationBufferMemory()

In [10]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})

In [11]:
print(memory.buffer)

Human: Hi
AI: What's up


In [12]:
from langchain.memory import ConversationBufferWindowMemory

In [13]:
memory = ConversationBufferWindowMemory(k=1) 

In [14]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

In [15]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [16]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    # verbose=True
    verbose=False,
)

In [17]:
conversation.predict(input="Hi, my name is Andrew")

"Hello Andrew! It's nice to meet you. How can I assist you today?"

In [18]:
conversation.predict(input="What is 1+1?")

'1+1 is equal to 2.'

In [19]:
conversation.predict(input="What is my name?")

"I'm sorry, but I don't have access to personal information about individuals unless it has been shared with me in the course of our conversation."

In [20]:
# ConversationTokenBufferMemory
# Limit by token
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0, model=llm_model)

In [21]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=40)
# Last
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
# First
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [22]:
memory.load_memory_variables({})

{'history': 'Human: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

In [23]:
# ConversationSummaryMemory
from langchain.memory import ConversationSummaryBufferMemory

In [24]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [25]:
memory.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings. The human asks about the schedule for the day. The AI provides a detailed schedule, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of bringing a laptop to showcase the latest LLM demo during the lunch meeting.'}

In [26]:
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=True
)

In [27]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI exchange greetings. The human asks about the schedule for the day. The AI provides a detailed schedule, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of bringing a laptop to showcase the latest LLM demo during the lunch meeting.
Human: What would be a good demo to show?
AI:

> Finished chain.


'A good demo to show during the lunch meeting with the customer interested in AI would be the latest LLM (Language Model) demo. The LLM is a cutting-edge AI model that can generate human-like text based on a given prompt. It has been trained on a vast amount of data and can generate coherent and contextually relevant responses. By showcasing the LLM demo, you can demonstrate the capabilities of AI in natural language processing and how it can be applied to various industries and use cases.'

In [28]:
memory.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings and discuss the schedule for the day. The AI provides a detailed schedule, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of bringing a laptop to showcase the latest LLM demo during the lunch meeting. The human asks what would be a good demo to show, and the AI suggests showcasing the latest LLM (Language Model) demo. The LLM is a cutting-edge AI model that can generate human-like text based on a given prompt. It has been trained on a vast amount of data and can generate coherent and contextually relevant responses. By showcasing the LLM demo, the human can demonstrate the capabilities of AI in natural language processing and how it can be applied to various industries and use cases.'}